# Assignment 3

## Instructions
- Run this notebook on ```Google Colab(preferable)```
- Write your code and analysis in the indicated cells.
- Ensure that this notebook runs without errors when the cells are run in sequence.
- Do not attempt to change the contents of other cells. 

## Packages Used
- sklearn [link](https://scikit-learn.org/)
- Keras [link](https://keras.io/guides/)

## Submission
- Rename the notebook to `<roll_number>_Assignment3_Q3.ipynb`.


## Question 3
Fake news is a widespread problem and there are many methods for combating it.
You have to build a fake news detection system using a ML model. Train any ML model (ANN, LSTM) over the given Dataset.
The dataset has short statements spoken by people and has the meta-information and corresponding label for those sentences. 
Your target is label column which has 6 labels(in the increasing order of truthfullness): pants-fire, false, barely-true, half-true, mostly-true, true.

The features are 'statement', 'subject', 'speaker', 'job', 'state', 'party', 'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c', 'pants_on_fire_c', 'venue' and the target is column "label".

The statement is made by speaker whose job, party are given along with 6 columns which are an account of the  type of news(labels) the person has shared before. 
The person who has shared fake content before is likely to share it in future and this can be accounted by the ML model as a feature. Column barely_true_c contains how many barely_true news has the speaker shared (and so is with column X_c, value of X_c is number of X the person shared).


You have to perform two tasks:
* task1: Binary classification <br>
Classify the given news as true/false. Take the labels pants-fire, false, barely-true as false and rest (half-true, mostly-true, true) as true.
* task2: Six-way classification <br>
Classify the given news into six-classes "pants-fire, false, barely-true, half-true, mostly-true, true".

For each of the tasks:
1) Experiment with depth of network and try to fine-tune hyperparameters reporting your observations. <br>
2) Report the accuracy, f1-score, confusion matrix on train, val and test sets. <br>
3) Experiment with bag-of-words, glove and bert embeddings(code given in the below notebook) and report results. <br> Comment on what is the affect of embedding on the results.

The pre-processing code is provided, you need to write the training and test.

Note: You are supposed to train on trainset, fine-tune on val and just eval on test set. If found that you trained on val/test sets, the penalty will be incurred.

In [1]:
# !pip install numpy
# !pip install tensorflow
# !pip install re
# !pip install nltk
# !pip install keras
# !pip install sklearn

In [2]:
# Importing libraries
import numpy as np
import pandas as pd
from tensorflow import keras  #feel free to use any other library
import numpy as np

import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score

import keras
from keras.models import Sequential,Model
from keras.layers import CuDNNLSTM, Dense, Dropout, LSTM,Input
from keras.optimizers import adam_v2
from keras.layers.merge import concatenate

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
train = pd.read_csv('/content/sample_data/train.csv')
val = pd.read_csv('/content/sample_data/val.csv')
test = pd.read_csv('/content/sample_data/test.csv')

In [4]:
# Dropping the 'id' column
train.drop('id', axis = 1, inplace = True)
test.drop('id', axis = 1, inplace = True)
val.drop('id', axis = 1, inplace = True)

In [5]:
train.head(5)

,label,statement,subject,speaker,job,state,party,barely_true_c,false_c,half_true_c,mostly_true_c,pants_on_fire_c,venue
0,False,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0,1,0,0,0,a mailer
1,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0,0,1,1,0,a floor speech.
2,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70,71,160,163,9,Denver
3,False,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7,19,3,5,44,a news release
4,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15,9,20,19,2,an interview on CNN


In [6]:
# Checking the shape of data
print(train.shape)
print(val.shape)
print(test.shape)

(10269, 13)
(1284, 13)
(1283, 13)


## Clean and pre-process data
* Replace missing values
* Remove numbers and special characters
* Convert to upper-case

We experiment with two types of processing, one directly appending the other attributes like subject, job, state, party to sentence and then applying bag of words on it.

Other being encoding sentence with glove embeddings and passing just that.

In [7]:

def dataPreprocessing(data):
    '''Function for cleaning the dataset
    '''
    corpus = []
    # Missing values
    data["job"].fillna("no-job", inplace = True)
    data["state"].fillna("no-state", inplace = True)

    for x in range(data.shape[0]):
        statement = re.sub('[^a-zA-Z]', ' ', data['statement'][x]) # Removing all numbers and special characters
        statement = statement.lower() # Converting uppercase to lowercase
        statement = statement.split()
        
        # you can experiment with any other stemmers
        ps = PorterStemmer()
        statement = [ps.stem(word) for word in statement if not word in set(stopwords.words('english'))] # Stemming the dataset and removing stopwords
        statement = ' '.join(statement)
        subject = data['subject'][x].replace(',', ' ')
        speaker = data['speaker'][x]
        job = data['job'][x].lower()
        # job = job.replace(' ', '-')
        state = data['state'][x].lower()
        party = data['party'][x].lower()
        corpus.append(statement + ' '  + subject + ' ' + job + ' ' + state + ' ' + party)
    return corpus

In [8]:
x_train = dataPreprocessing(train)
x_val = dataPreprocessing(val) 
x_test = dataPreprocessing(test) 

In [9]:
len(x_train), len(x_val), len(x_test)

(10269, 1284, 1283)

In [10]:
corpus = x_train + x_val + x_test

## Using bag-of-words embedding


In [11]:
# Converting the corpus into bag-of-words
cv = CountVectorizer(max_features = 8000)
X = cv.fit_transform(corpus).toarray()

In [12]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [13]:
X.shape

(12836, 8000)

In [14]:
train.columns

Index(['label', 'statement', 'subject', 'speaker', 'job', 'state', 'party',
       'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c',
       'pants_on_fire_c', 'venue'],
      dtype='object')

In [15]:
# Selecting the columns 'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c'
label_cols = ['barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c',
       'pants_on_fire_c']
x_train2 = train[label_cols]
x_val2 = val[label_cols]
x_test2 = test[label_cols]

In [16]:
print(x_val2)
print(X[:len(x_train)])

      barely_true_c  false_c  half_true_c  mostly_true_c  pants_on_fire_c
0                 1        0            1              0                0
1                11       43            8              5              105
2                 0        1            1              1                0
3                 0        1            1              1                0
4                70       71          160            163                9
...             ...      ...          ...            ...              ...
1279             70       71          160            163                9
1280             40       29           69             76                7
1281              0        1            0              2                0
1282              1        4            4              1                0
1283              9       11           10              7                3

[1284 rows x 5 columns]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 

In [17]:
# Stacking x_train and x_train2 horizontally
x_train_bow = np.hstack((X[:len(x_train)], x_train2))
x_val_bow = np.hstack((X[len(x_train):len(x_train)+len(x_val)], x_val2))
x_test_bow = np.hstack((X[len(x_train)+len(x_val):], x_test2))

In [18]:
x_train_bow.shape

(10269, 8005)

## Use of Glove Embedding


download glove embeddings from 'https://nlp.stanford.edu/data/glove.6B.zip','glove.6B.zip'
and place in your current working folder


In [19]:
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip "glove.6B.zip" -d "glove"

--2022-04-13 17:15:58--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-04-13 17:15:58--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.2’

glove.6B.zip.2      100%[===================>] 822.24M  5.09MB/s    in 2m 40s  

2022-04-13 17:18:38 (5.14 MB/s) - ‘glove.6B.zip.2’ saved [862182613/862182613]

Archive:  glove.6B.zip
replace glove/glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace glove

In [20]:
emmbed_dict = {}
with open('glove/glove.6B.200d.txt','r') as f:
  for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:],'float32')
    emmbed_dict[word]=vector


In [21]:
emmbed_dict['oov'] = np.zeros(200)

In [22]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [23]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')
def dataPreprocessing_glove(data):
    corpus = []
    # Missing values
    data["job"].fillna("no-job", inplace = True)
    data["state"].fillna("no-state", inplace = True)

    for x in range(data.shape[0]):
        statement = re.sub('[^a-zA-Z]', ' ', data['statement'][x]) # Removing all numbers and special characters
        statement = statement.lower() # Converting uppercase to lowercase
        statement = word_tokenize(statement)

        embed_statement = []
        for w in statement:
            if w in emmbed_dict:
                embed_statement.append(emmbed_dict[w])
            else:
                embed_statement.append(emmbed_dict['oov'])
         
        # bonus: Think how you can encode the below features(hint: look upon label encoding or training your own word2vec or any other embedding model)
    
#         subject = data['subject'][x].replace(',', ' ')
#         speaker = data['speaker'][x]
#         job = data['job'][x].lower()
#         # job = job.replace(' ', '-')
#         state = data['state'][x].lower()
#         party = data['party'][x].lower()
        corpus.append(embed_statement)
    
    return np.array(corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [24]:
x_train_glove = dataPreprocessing_glove(train)
x_val_glove = dataPreprocessing_glove(val) 
x_test_glove = dataPreprocessing_glove(test) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [25]:
x_train_glove = np.hstack((x_train_glove.reshape(-1,1), x_train2))
x_val_glove = np.hstack((x_val_glove.reshape(-1,1), x_val2))
x_test_glove = np.hstack((x_test_glove.reshape(-1,1), x_test2))



In [26]:
print(x_train_glove.shape)
print(len(x_train_glove[0][0]))

(10269, 6)
12


## Use of bert embeddings
note: we used our pre-processed code for bow which has the attributed appended to end the end of sentence. 

In [27]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

x_train_bert = np.hstack((model.encode(x_train), x_train2))
x_val_bert = np.hstack((model.encode(x_val), x_val2))
x_test_bert = np.hstack((model.encode(x_test), x_test2))

#x_train_bert = model.encode(x_train)
#x_val_bert = model.encode(x_val)
#x_test_bert = model.encode(x_test)

In [28]:
print(x_train_bow.shape)
print(x_train_bert.shape)
print(x_train_glove.shape)
print(type(x_train_bow))
print(len(x_train_glove[0][0][1]),len(x_train_glove[1][0][0]),len(x_train_glove[2][0][1]))

(10269, 8005)
(10269, 389)
(10269, 6)
<class 'numpy.ndarray'>
200 200 200


Now use the above 3 types of embedded inputs(bow, glove, bert embeddings) for the 2 classification tasks and compare their outputs


# Six-way classification

## Preprocessing

In [29]:
num_classes = 6
# Preprocessing function for the labels
def categorize(data):
    y = data["label"].tolist()

    # Encoding the Dependent Variable
    labelencoder_y = LabelEncoder()
    y = labelencoder_y.fit_transform(y)

    # Converting to binary class matrix
    y = np_utils.to_categorical(y, num_classes)
    return y

In [30]:
y_train_six_way = categorize(train)
y_test_six_way = categorize(test)
y_val_six_way = categorize(val)

In [31]:
print(y_train_six_way.shape)

(10269, 6)


Build a model and pass bow, glove and bert embedded inputs: x_train_bow, x_train_glove, x_train_bert(similarly validate for val and report results on test)


## Model

In [32]:
## write your code here

# Initialize hyperparameters
batch_size=32
units=128
drop_out=0.1
learning_rate = 0.01
decay=1e-6
epoch=5
embedding_size_bag=8005
embedding_size_bert=389

# Create model
"""
visible1 = Input(shape=(embedding_size,1)) # for each batch size
lstmLayer = LSTM(units=units)(visible1)
visible2 = Input(shape=(5,))
merge = concatenate([visible2, lstmLayer])
#merge1=Dense(32, activation='relu')(merge)
output = Dense(6, activation='softmax')(merge)
model = Model(inputs=[visible1, visible2], outputs=output)
print(model.summary())
"""
optimizer = adam_v2.Adam(learning_rate=learning_rate,decay=decay)

# Simple Nueral net model
def models(embedding_size,x_train, y_train_six_way1,x_val,sixclass=True, val1=[]):
  NNmodel = Sequential()
  NNmodel.add(Dense(1024, input_dim=embedding_size, activation='relu'))
  NNmodel.add(Dense(256, activation='relu'))
  NNmodel.add(Dense(126, activation='relu'))
  NNmodel.add(Dense(64, activation='relu'))
  if sixclass:
    NNmodel.add(Dense(6, activation='softmax'))
  else:
    NNmodel.add(Dense(2, activation='softmax'))
  print(NNmodel.summary())

  NNmodel.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])
  # train
  if sixclass:
    NNmodel.fit(x_train, y_train_six_way1, epochs=epoch, batch_size=batch_size, shuffle=True,validation_data=(x_val,y_val_six_way))
  else:
    NNmodel.fit(x_train, y_train_six_way1, epochs=epoch, batch_size=batch_size, shuffle=True,validation_data=(x_val,val1))
  return NNmodel

"""
categorical_crossentropy (cce) produces a one-hot array containing the probable match for each category,
sparse_categorical_crossentropy (scce) produces a category index of the most likely matching category.
"""

# test
# report accuracy, f1-score and confusion matrix

def confusion_utility(x_test,y_train_six_way,NNmodel):
  y_test=[]
  y_pred=[]
  for i in range(len(x_test)):
    pred = NNmodel.predict(x_test[i:i+1])
    a=int(np.argmax(pred[0]))
    b=int(np.argmax(y_train_six_way[i]))
    y_pred.append(a)
    y_test.append(b)
  return y_test,y_pred


**Bag of word embedding on 6 way classification**

In [33]:
NNmodel=models(embedding_size_bag,x_train_bow,y_train_six_way,x_val_bow)
y_test,y_pred=confusion_utility(x_test_bow,y_test_six_way,NNmodel)
confusion = confusion_matrix(y_test, y_pred)
print('Confusion Matrix\n')
print(confusion)
print('\nAccuracy: ',accuracy_score(y_test, y_pred))
print('Micro F1-score: ',f1_score(y_test, y_pred, average='micro'))
print('Macro F1-score: ',f1_score(y_test, y_pred, average='macro'))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              8198144   
                                                                 
 dense_1 (Dense)             (None, 256)               262400    
                                                                 
 dense_2 (Dense)             (None, 126)               32382     
                                                                 
 dense_3 (Dense)             (None, 64)                8128      
                                                                 
 dense_4 (Dense)             (None, 6)                 390       
                                                                 
Total params: 8,501,444
Trainable params: 8,501,444
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
321/321 [======================

**BERT Embedding on 6 way classifications**

In [34]:
NNmodel=models(embedding_size_bert,x_train_bert,y_train_six_way,x_val_bert)
y_test,y_pred=confusion_utility(x_test_bert,y_test_six_way,NNmodel)

confusion = confusion_matrix(y_test, y_pred)
print('Confusion Matrix\n')
print(confusion)

print('\nAccuracy: ',accuracy_score(y_test, y_pred))
print('Micro F1-score: ',f1_score(y_test, y_pred, average='micro'))
print('Macro F1-score: ',f1_score(y_test, y_pred, average='macro'))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 1024)              399360    
                                                                 
 dense_6 (Dense)             (None, 256)               262400    
                                                                 
 dense_7 (Dense)             (None, 126)               32382     
                                                                 
 dense_8 (Dense)             (None, 64)                8128      
                                                                 
 dense_9 (Dense)             (None, 6)                 390       
                                                                 
Total params: 702,660
Trainable params: 702,660
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
321/321 [========================

**Glove encoding on 6 way Classification**

# Binary Classification

## Preprocessing

In [35]:
num_classes = 2

In [36]:
# Function for preprocessing labels
def dataPreprocessingBinary(data):
    y = data["label"].tolist()

    # Changing the 'half-true', 'mostly-true', barely-true', 'pants-fire' labels to True/False for Binary Classification
    for x in range(len(y)):
        if(y[x] == 'half-true'):
            y[x] = 'True'
        elif(y[x] == 'mostly-true'):
            y[x] = 'True'
        elif(y[x] == 'barely-true'):
            y[x] = 'False'
        elif(y[x] == 'pants-fire'):
            y[x] = 'False'

    # Converting the lables into binary class matrix
    labelencoder_y = LabelEncoder()
    y = labelencoder_y.fit_transform(y)
    y = np_utils.to_categorical(y, num_classes)
    return y

In [37]:
y_train_binary = dataPreprocessingBinary(train)
y_test_binary = dataPreprocessingBinary(test)
y_val_binary = dataPreprocessingBinary(val)

## Model
Build a model and pass bow, glove and bert embedded inputs: x_train_bow, x_train_glove, x_train_bert(similarly validate for val and report results on test)


**Bag of word embedding**

In [38]:
## write your code here
# Initialize hyperparameters
# Create model
# train
# test
# report accuracy, f1-score and confusion matrix

NNmodel=models(embedding_size_bag,x_train_bow,y_train_binary,x_val_bow,False,y_val_binary)
y_test,y_pred=confusion_utility(x_test_bow,y_train_binary,NNmodel)
confusion = confusion_matrix(y_test, y_pred)
print('Confusion Matrix\n')
print(confusion)
print('\nAccuracy: ',accuracy_score(y_test, y_pred))
print('Micro F1-score: ',f1_score(y_test, y_pred, average='micro'))
print('Macro F1-score: ',f1_score(y_test, y_pred, average='macro'))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 1024)              8198144   
                                                                 
 dense_11 (Dense)            (None, 256)               262400    
                                                                 
 dense_12 (Dense)            (None, 126)               32382     
                                                                 
 dense_13 (Dense)            (None, 64)                8128      
                                                                 
 dense_14 (Dense)            (None, 2)                 130       
                                                                 
Total params: 8,501,184
Trainable params: 8,501,184
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
321/321 [====================

**Bert Embedding**

In [39]:
NNmodel=models(embedding_size_bert,x_train_bert,y_train_binary,x_val_bert,False,y_val_binary)
y_test,y_pred=confusion_utility(x_test_bert,y_train_binary,NNmodel)
confusion = confusion_matrix(y_test, y_pred)
print('Confusion Matrix\n')
print(confusion)
print('\nAccuracy: ',accuracy_score(y_test, y_pred))
print('Micro F1-score: ',f1_score(y_test, y_pred, average='micro'))
print('Macro F1-score: ',f1_score(y_test, y_pred, average='macro'))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 1024)              399360    
                                                                 
 dense_16 (Dense)            (None, 256)               262400    
                                                                 
 dense_17 (Dense)            (None, 126)               32382     
                                                                 
 dense_18 (Dense)            (None, 64)                8128      
                                                                 
 dense_19 (Dense)            (None, 2)                 130       
                                                                 
Total params: 702,400
Trainable params: 702,400
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
321/321 [========================

**Difference between Encodings on classification**

* Binary Classifier are performing better than 6 way classification problems. It is intuative as well, becuase at random binary have probability of 0.5 whereas 6 have have 0.17 probability.
* Bert takes less space and performs equally well in case of binary classifier. I am not able to exploit the sequence in Bert by applying LSTM, in that case it can perform better.
* For 6 way classifer, Bag of word is performing better than Bert. Since I have not used sequence of encodings well. In general Bert perform a better encoding than Bag of words.